In [7]:
import requests
import time
import json
from datetime import datetime
from requests_oauthlib import OAuth1

credentials = {
    'CONSUMER_KEY': '',
    'CONSUMER_SECRET': '',
    'TOKEN_KEY': '',
    'TOKEN_SECRET': '',
}

def authenticate(credentials):
    try:
        oauth = OAuth1(client_key=credentials['CONSUMER_KEY'],
                      client_secret=credentials['CONSUMER_SECRET'],
                      resource_owner_key=credentials['TOKEN_KEY'],
                      resource_owner_secret=credentials['TOKEN_SECRET'],
                      signature_type='auth_header')
        client = requests.session()
        client.auth = oauth
        return client
    except (KeyError, TypeError):
        print('Error setting auth credentials.')
        raise

client = authenticate(credentials)

URL = 'https://api.twitter.com/1.1/search/tweets.json'
QUERY = '(#Romaine OR @Romaine) AND -filter:retweets' #Change for SaudiArabia and Thanksgiving
MAX_TWEETS = 10_000

statuses = []
flag = True
params = {'q': QUERY, 'count': 100, 'tweet_mode': 'extended'}


In [8]:

while flag:
    response = client.get(URL, params=params)
    
    if response.status_code == 200:
        payload = response.json()
        statuses.append(payload['statuses'])
        metadata = payload['search_metadata']
        
        if 'next_results' in metadata:
            params['max_id'] = metadata['next_results'].split('?')[1].split('&')[0].split('=')[1]
        else:
            flag = False
            continue
            
        print('Got {0} statuses, sleeping for 5 seconds'.format(len(statuses) * 100));
        time.sleep(5)
    else:
        print('Sleeping for 10 seconds, status is:{0}'.format(response.status_code));
        time.sleep(10)
   
    if len(statuses) * 100 >= MAX_TWEETS:
        flag = False
        continue

Got 100 statuses, sleeping for 5 seconds
Got 200 statuses, sleeping for 5 seconds
Got 300 statuses, sleeping for 5 seconds
Got 400 statuses, sleeping for 5 seconds
Got 500 statuses, sleeping for 5 seconds
Got 600 statuses, sleeping for 5 seconds


In [9]:
#Change for SaudiArabia and Thanksgiving
with open('path\\Romaine.json', 'w') as outfile:
    json.dump(statuses, outfile)